In [1]:
import torch
import json
from embed_llm.models.augmented_model import EmbedAugPipeline

/home/hippolytepilchen/micromamba/envs/llm_embed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm_path = '/lustre/scwpod02/client/kyutai-interns/hippop/models/mistral_7B'
#Must have a params json for pipeline

# No embeddings:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/no_embed_bs16_lr5e-5Mistral7B88d0b42410aa4ec12025/checkpoints/checkpoint_002500'

# Length tokens:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_512t_Mistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_005000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_256t_Mistral7Be9ffc00fa42bedbc50d0/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_128t_Mistral7B226729d875c65b331ef8/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_64t_Mistral7B9bbea1b3b8dc23079b04/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_32t_Mistral7Bccbc3f29d69bd124c6cf/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/SL_16t_Mistral7B7bc7dcc2ba28873eda96/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/mean_not_causal/checkpoints/checkpoint_007500'


# # Continuation:
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/continuation_Mistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_006000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/mean_finetuned_notcausal_continuationMistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_005500'

# # Cross-Attention:
ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_5_last_layersMistral7Bdbbb7faebb2f32cf20e9/checkpoints/checkpoint_010000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_fine_tuned_embedder_5_last_layersMistral7Bdbbb7faebb2f32cf20e9/checkpoints/checkpoint_007500'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_finetuned_notcausal_continuationMistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_005000'
# ckpt_path = '/lustre/scwpod02/client/kyutai-interns/hippop/tmp/cross_att_pretrained_continuationMistral7B20ed0018b2a84fba09c4/checkpoints/checkpoint_008500'


model_name = 'Mistral7B' # Mistral7B, Llama3.2-3B, Gemma7B
embed_model_name = 'NVEmbed' #'NVEmbed'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
w_embeds = True


# variant = '7b' if model_name == 'Gemma7B' else None

In [3]:
pipeline: EmbedAugPipeline = EmbedAugPipeline.load_inference_model(llm_path = llm_path, 
                                                                   ckpt_path = ckpt_path, 
                                                                   device = device,
                                                                   llm_name = model_name, 
                                                                   embed_model_name = embed_model_name,
                                                                    max_batch_size = 4)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.12s/it]


Not only LoRA weights found in the checkpoint. Skipping other weights.
Loading and merging LoRA weights...


In [4]:
# 1 information in the doc which enables to answer the question but not good response often in-context
# 2 information in the doc which enables to answer the question and good response often in-context
# 3 Hard negative passage
# 4 Same

# prompt_prefix = "Query: who wrote the song photograph by ringo starr\nAnswer: Ringo Starr\n\nQuery: who is playing the halftime show at super bowl 2016\nAnswer: Coldplay\n\nQuery: where was the world economic forum held this year\nAnswer: Davos\n\nQuery: where are the giant redwoods located in california\nAnswer: Humboldt County\n\nQuery: who has made the most premier league appearances\nAnswer: Gareth Barry\n\nQuery: "
# prompts = ['who has most followers on instagram in world','who did the united states win its independence from', 'locations for the film an englishman who went up a hill', 'who is the valley of the dolls based on']
# final_prompts = [prompt_prefix + prompt + '\nAnswer:' for prompt in prompts]

# text_conditioning = ["This list contains the top 50 accounts with the most followers on the photo and video-sharing social platform Instagram. As of July 2019, the most followed user is Instagram's own account, with over 308 million followers. Cristiano Ronaldo is the most followed individual, with over 177 million followers. Fifteen accounts have exceeded 100 million followers on the site.",
#                      "During the American Revolution, the legal separation of the thirteen colonies from Great Britain in 1776 actually occurred on July 2, when the Second Continental Congress voted to approve a resolution of independence that had been proposed in June by Richard Henry Lee of Virginia declaring the United States independent from Great Britain's rule. After voting for independence, Congress turned its attention to the Declaration of Independence, a statement explaining this decision, which had been prepared by a Committee of Five, with Thomas Jefferson as its principal author. Congress debated and revised the wording of the Declaration, finally approving it two days later on July 4. A day earlier, John Adams had written to his wife Abigail",
#                      'The village was a primary location for the making of the film \"The Englishman Who Went Up a Hill But Came Down a Mountain\", which starred Hugh Grant. The hilltop scenes were filmed on the Gyrn, the long hill that overlooks the village. It was also featured in \"Monk\'s Hood\", an episode of \"The Cadfael Chronicles\"',
#                      'Valley of the Dolls is the first novel by American writer Jacqueline Susann. Published in 1966, the book was the biggest selling novel of its year. To date, it has sold more than 31 million copies, making it one of the best-selling works in publishing history.']

# answers = ['Instagram','Great Britain',"Llansilin in Powys",["Judy Garland", "Carole Landis", "Dean Martin", "Ethel Merman"]]

n_passages = 4
eval_data = '/lustre/scwpod02/client/kyutai-interns/datasets/modular_finetuning/enwiki-20220120_valid.jsonl'
train_data = '/lustre/scwpod02/client/kyutai-interns/datasets/modular_finetuning/enwiki-20220120_train.jsonl'
train_passage = []
valid_passage = []
with open(train_data, 'r') as f:
    for i, line in enumerate(f):
        if i == n_passages:
            break
        train_passage.append(json.loads(line)['text'].split('\n\n')[1])
        
with open(eval_data, 'r') as f:
    for i, line in enumerate(f):
        if i == n_passages:
            break
        valid_passage.append(json.loads(line)['text'].split('\n\n')[1])
# print('Train passage:', train_passage)
# print('Valid passage:', valid_passage)

In [5]:
w_embeds = True
max_tokens = 200
temperature = 0

tests = [{'w_embeds': True, 'temperature': 0 },  {'w_embeds': True, 'temperature': 0.7 }, {'w_embeds': False, 'temperature': 0.7 }]

In [6]:
pipeline.pipeline_args.w_embeds = True
generated_sequence = pipeline.generate(prompts = ['who has most followers on Instagram in world?'],
                                      text_conditioning = ["This list contains the top 50 accounts with the most followers on the photo and video-sharing social platform Instagram. As of July 2019, the most followed user is Instagram's own account, with over 308 million followers. Cristiano Ronaldo is the most followed individual, with over 177 million followers."],
                                      temperature = 0.4, 
                                      max_tokens =200,
                                      truncate_double_space = False)
print(generated_sequence)

/home/hippolytepilchen/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/5130cf1daf847c1bacee854a6ef1ca939e747fb2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),


["\n\nThe list contains the top 50 accounts with the most followers on the photo-sharing social media platform Instagram. As of July 2019, the most followed user is Cristiano Ronaldo, with over 308 million followers. Instagram's own account, @instagram, is the most followed account, with over 407 million followers."]


## Reconstruction

In [ ]:
for param in tests:
    print('Param:', param)
    if param['w_embeds']:
        pipeline.pipeline_args.w_embeds = True
    else:
        pipeline.pipeline_args.w_embeds = False
    
    final_valid_prompts = [passage.split(' ')[0] for passage in valid_passage][2] 
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    print('Prompt', final_valid_prompts, ' | Passage', text_valid_conditioning)
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  word', generated_sequence)
    
    final_valid_prompts = ['' for passage in train_passage][1]
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    print('Prompt', final_valid_prompts, ' | Passage', text_valid_conditioning)
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  empty', generated_sequence)

    final_train_prompts =  [passage.split(' ')[0] for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    print('Prompt', final_train_prompts, ' | Passage', text_train_conditioning)
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train word', generated_sequence)
    final_train_prompts = ['' for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    print('Prompt', final_train_prompts, ' | Passage', text_train_conditioning)
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train empty', generated_sequence)

    

Param: {'w_embeds': True, 'temperature': 0}
Prompt The  | Passage The Roman Republic (Repubblica Romana) was a sister republic of the First French Republic. It was pr
Valid  word ['Roman Republic (Repubblica romana)\n\nThe Roman Republic (Repubblica romana) was a sister republic of the First French Republic. It was pr.']
Prompt   | Passage The Roman Republic (Repubblica Romana) was a sister republic of the First French Republic. It was pr
Valid  empty ['Roman Republic (Repubblica Romana)\n\nThe Roman Republic (Repubblica Romana) was a sister republic of the First French Republic. It was pr.']
Prompt Cochamó  | Passage Cochamó is a Chilean town and commune located in Llanquihue Province, Los Lagos Region. The capital 
Train word ['is a Chilean town and commune located in Llanquihue Province, Los Lagos Region. The capital  of  Llanquihue Region, Los Lagos.']
Prompt   | Passage Cochamó is a Chilean town and commune located in Llanquihue Province, Los Lagos Region. The capital 
Train empty

In [ ]:
# Continuation
for param in tests:
    print('Param:', param)
    if param['w_embeds']:
        pipeline.pipeline_args.w_embeds = True
    else:
        pipeline.pipeline_args.w_embeds = False
    
    final_valid_prompts = [passage[100:].split(' ')[0] for passage in valid_passage][2] 
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    print('Passage', text_valid_conditioning, ' | Truth', [passage[100:200] for passage in valid_passage][2] )
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  word', generated_sequence)
    
    final_valid_prompts = ['' for passage in train_passage][1]
    text_valid_conditioning = [passage[:100] for passage in valid_passage][2]
    generated_sequence = pipeline.generate(prompts = final_valid_prompts, 
                                        text_conditioning = text_valid_conditioning, 
                                        temperature = param['temperature'], 
                                        max_tokens = max_tokens,
                                        truncate_double_space = False)
    print('Valid  empty', generated_sequence)

    final_train_prompts =  [passage[100:].split(' ')[0] for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    print('Passage', text_train_conditioning, ' | Truth', [passage[100:200] for passage in train_passage][1] )
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train word', generated_sequence)
    final_train_prompts = ['' for passage in train_passage][1] 
    text_train_conditioning = [passage[:100] for passage in train_passage][1]
    generated_sequence = pipeline.generate(prompts = final_train_prompts, 
                                       text_conditioning = text_train_conditioning, 
                                       temperature = param['temperature'], 
                                       max_tokens = max_tokens,
                                       truncate_double_space = False)
    print('Train empty', generated_sequence)


Param: {'w_embeds': True, 'temperature': 0}
Passage The Roman Republic (Repubblica Romana) was a sister republic of the First French Republic. It was pr  | Truth oclaimed on 18 February 1798 after Louis-Alexandre Berthier, a general of Napoleon, had occupied the
Valid  word ['sister republic of the First French Republic) was a sister republic of the First French Republic. It was romp']
Valid  empty ['Roman Republic (Repubblica Romana)\n\nThe Roman Republic (Repubblica Romana) was a sister republic of the First French Republic. It was pr.']
Passage Cochamó is a Chilean town and commune located in Llanquihue Province, Los Lagos Region. The capital   | Truth of the commune is the town of Río Puelo, which is named after the Puelo River.
Train word ['Cochamó is a Chilean town and commune located in Llanquihue Province, Los Lagos Region. The capital  of  Llanquihue Region, Los Lagos.']
Train empty ['Cochamó is a Chilean town and commune located in Llanquihue Province, Los Lagos Region. The c

UnboundLocalError: local variable 'kv_seqlens' referenced before assignment